## Main code

### Libraries and Modules

In [1]:
import sys
import math
import glob
import os
import time
import csv

import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import Voronoi
import cv2

import import_ipynb
import voronoi_constructor as bvc_builder

importing Jupyter notebook from voronoi_constructor.ipynb
importing Jupyter notebook from node_linker.ipynb
importing Jupyter notebook from solver.ipynb


### Global Parameters

In [9]:
num_robots = 5
parameters = {"radius"   : 1,
              "movement" : 1,
              "tolerance": 2,
              "deadlock_mov" : 10,
              "previous" : 1
             }

### General Options

In [3]:
# Initialize the number of infinity vertices
infinity_vertices = 0
# To colorize each robots and its related data
palette = bvc_builder.colors(num_robots)
# Allow or deny (True/False) optimization
optimize = False
# True to plot figures
plotting = False
# True to save the image in hard disc
save_image = False
# True to register data in CSV file
save_data = True
# True to check the execution of the main module
main_verbose = False
# True to check the execution of all code
verbose = False
experiments = 30
seed = 0

### Main Loop

In [6]:
def main_module(key):
    with open("/home/leduin/Desktop/Hub/Results/aga_"+key+".csv","a") as file:
        if save_data:
            csv_writer = csv.writer(file)
        for experiment in range(experiments):
            # INITIALIZATION
            start_time = time.time()
            robots, target, distances = bvc_builder.position_generator(num_robots,
                                                            experiment+seed)
            vor = Voronoi(robots) # Generate Voronoi Diagram using Scipy module
            vor_target = Voronoi(target)
            robots_path = [] # To save the robots path along execution
            finished = np.all(robots == target) # Flag to stop the main loop
            # To save the traveled distance of each robot 
            traveled_distances = [0 for _ in range(num_robots)]
            counter = 0 # For steps
            # COLLISION FREE CONFIGURATION
            if not bvc_builder.collision_free_conf(robots, vor.ridge_points,
                                                   parameters["radius"]):
                print('ERROR!: Change the initial configuration')
                print("Positions:", vor.ridge_points)
                break
            if not bvc_builder.collision_free_conf(target,
                                                   vor_target.ridge_points,
                                                   parameters["radius"]):
                print('ERROR!: Change the final configuration')
                print("Positions:", vor_target.ridge_points)
                break
            # MAIN LOOP
            while not finished:
                #Final iteration
                if np.all(robots == target):
                    finished = True
                if main_verbose:
                    print('Robots:\n', robots,'\n')
                # Generate Voronoi Diagram for the current positions
                vor = Voronoi(robots)
                # Generate the Voronoi Diagram and get the approximated values
                # of inf vertices
                vertices, edges = bvc_builder.voronoi_approx(robots, vor,
                                                             verbose)
                # Assemble the vertices and get the ordered vertices
                bvc = bvc_builder.vertices_assembler(robots, vor, edges,
                                                     verbose)
                robots_path.append(robots) # Save the previous position
                # Generate the BVC
                closer_points,vertices_bvc,closer_edges = bvc_builder.bvc_generator(
                    robots, target, parameters["radius"], vertices, edges,
                    bvc, robots_path, optimize, plotting, verbose)
                # SAVING IMAGE
                if save_image:
                    folder = 'Simple'
                    if optimize:
                        folder = 'Opt'
                    plt.savefig('/home/leduin/Desktop/Hub/Images/'+folder+'/'
                                +str(num_robots)+'/fig'+ str(counter)+'.svg')
                    print('Image '+str(counter)+' saved successfully!')
                    plt.close()
                elif plotting:
                    plt.show()
                # DEADLOCK TESTING
                deadlock_flag = False
                deadlock_pointers = []
                if counter >= parameters["previous"]:
                    for i in range(num_robots):
                        if not np.all(closer_points[i] == target[i]):
                            j = 0
                            deadlock_local_flag = False
                            distance1 = np.linalg.norm(closer_points[i]
                                                           - robots[i])
                            if main_verbose:
                                print('Distance from',palette[i],
                                      'to the closest point:',distance1)
                            while (not deadlock_local_flag
                                   and j < parameters["previous"]):
                                distance2 = np.linalg.norm(closer_points[i]
                                                           - robots_path[-2-j][i])
                                if main_verbose:
                                    print('Distance from the '+str(j+1)+
                                          'th previous point:', distance2)
                                if (distance1 < parameters["tolerance"]
                                    or distance2 < parameters["tolerance"]):
                                    deadlock_pointers.append(i)
                                    deadlock_local_flag = True
                                    deadlock_flag = True
                                j += 1
                if main_verbose:
                    print('Closer positions:', closer_points)
                    print('Deadlock?:',deadlock_flag,'\n')
                # BREAKING DEADLOCK
                if deadlock_flag:
                    if main_verbose:
                        print('Cells in Deadlock:')
                        for i in deadlock_pointers:
                            print(palette[i], end = ' ')
                        print('\nCloser_edges:', closer_edges, '\n')
                    closer_points = bvc_builder.deadlock(robots,closer_points,
                                                         vertices_bvc,
                                                         parameters["deadlock_mov"]*0.1,
                                                         robots_path,
                                                         deadlock_pointers,
                                                         parameters["tolerance"],
                                                         closer_edges,
                                                         parameters["previous"],
                                                         parameters["movement"],
                                                         verbose)
                    if main_verbose:
                        print('DEADLOCK ABOVE!')
                        print('Previous positions:', robots_path[-2])
                        print('New closer positions:', closer_points,'\n')
                if main_verbose:
                    print('\nRounds: ',counter)
                    print('\n*-*-*-*-*-*-*-*ENDING ROUND*-*-*-*-*-*-*-*-*\n')
                counter += 1
                # UPDATING POSITIONS
                internal_paths = closer_points - robots
                new_positions = []
                for i in range(num_robots):
                    if np.all(internal_paths[i] == 0):
                        # Paralyzed Robot for deadlock
                        if i in deadlock_pointers: 
                            new_positions.append(closer_points[i])
                        else: # Reached target
                            new_positions.append(target[i])
                    else:
                        candidate = (robots[i] +
                                     ((internal_paths[i]
                                       / np.linalg.norm(internal_paths[i]))
                                      * parameters["movement"]))
                        # Reaching the final position
                        if bvc_builder.isBetween(robots[i], candidate,
                                                 target[i]):
                            new_positions.append(target[i])
                            traveled_distances[i] += np.linalg.norm(robots[i]
                                                                    - target[i])
                        # Internal Path less than alpha
                        elif (np.linalg.norm(internal_paths[i])
                              < parameters["movement"]):
                            new_positions.append(closer_points[i])
                            traveled_distances[i] += np.linalg.norm(internal_paths[i])
                        # Normal movement
                        else:
                            new_positions.append(candidate)
                            traveled_distances[i] += parameters["movement"]
                robots = np.array(new_positions)
            effectiveness = np.sum(distances) / np.sum(traveled_distances)
            if save_data:
                csv_writer.writerow([parameters[key], experiment+seed,
                                     counter, time.time() - start_time,
                                     effectiveness])
                #print('Data of experiment ' + str(experiment+seed) +
                #      ' saved successfully!\n')

### One Parameter Analysis

In [10]:
parameter = "tolerance"
print("Working on",parameter,"...")
previous_value = parameters[parameter]
if parameter == "deadlock_mov":
    parameters[parameter] = 1
for i in range(10):
    print(parameter,"equal to",parameters[parameter],"...")
    main_module(parameter)
    parameters[parameter] += 1
parameters[parameter] = previous_value

Working on tolerance ...
tolerance equal to 2 ...
tolerance equal to 3 ...
tolerance equal to 4 ...
tolerance equal to 5 ...
tolerance equal to 6 ...
tolerance equal to 7 ...
tolerance equal to 8 ...
tolerance equal to 9 ...
tolerance equal to 10 ...
tolerance equal to 11 ...


### All Parameter Analysis

In [5]:
for parameter in parameters:
    print("Working on",parameter,"...")
    previous_value = parameters[parameter]
    if parameter == "deadlock_mov":
        parameters[parameter] = 1
    for i in range(10):
        print(parameter,"equal to",parameters[parameter],"...")
        main_module(parameter)
        parameters[parameter] += 1
    parameters[parameter] = previous_value

Working on radius ...
radius equal to 1 ...


<string>:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<string>:16: RuntimeWarning: invalid value encountered in double_scalars


radius equal to 2 ...
radius equal to 3 ...
radius equal to 4 ...
radius equal to 5 ...
radius equal to 6 ...
radius equal to 7 ...
radius equal to 8 ...
radius equal to 9 ...
radius equal to 10 ...
Working on movement ...
movement equal to 1 ...
movement equal to 2 ...
movement equal to 3 ...
movement equal to 4 ...
movement equal to 5 ...
movement equal to 6 ...
movement equal to 7 ...
movement equal to 8 ...
movement equal to 9 ...
movement equal to 10 ...
Working on tolerance ...
tolerance equal to 1 ...
tolerance equal to 2 ...
tolerance equal to 3 ...
tolerance equal to 4 ...
tolerance equal to 5 ...
tolerance equal to 6 ...
tolerance equal to 7 ...
tolerance equal to 8 ...
tolerance equal to 9 ...
tolerance equal to 10 ...
Working on deadlock_mov ...
deadlock_mov equal to 1 ...
deadlock_mov equal to 2 ...
deadlock_mov equal to 3 ...
deadlock_mov equal to 4 ...
deadlock_mov equal to 5 ...
deadlock_mov equal to 6 ...
deadlock_mov equal to 7 ...
deadlock_mov equal to 8 ...
deadlock

### Generate Video

In [11]:
frameSize = (432, 288)

out = cv2.VideoWriter('/home/leduin/Desktop/Hub/Images/Simple/animation_simple_'
                      +str(num_robots)+'.mp4',
                      cv2.VideoWriter_fourcc(*'DIVX'), 4, frameSize)

for filename in sorted(glob.glob('/home/leduin/Desktop/Hub/Images/Simple/'
                                 +str(num_robots)+'/*.png'),
                       key=os.path.getmtime):
    img = cv2.imread(filename)
    out.write(img)

out.release()